In [3]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm

def convert_pcap_to_csv(input_file, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, os.path.splitext(os.path.basename(input_file))[0] + '.csv')
    
    command = f"cicflowmeter -f {input_file} -c {output_file}"
    
    try:
        subprocess.run(command, shell=True, check=True)
        print(f"Successfully converted {input_file} to {output_file}")
        return output_file
    except subprocess.CalledProcessError as e:
        print(f"Error converting {input_file}: {e}")
        return None

def process_pcap_files(input_directory, output_directory):
    print("Processing PCAP files")
    csv_files = []
    
    for root, _, files in os.walk(input_directory):
        for file in tqdm(files, desc="Converting PCAP files"):
            if file.endswith('.pcap'):
                input_file = os.path.join(root, file)
                converted_file = convert_pcap_to_csv(input_file, output_directory)
                if converted_file:
                    csv_files.append(converted_file)
    
    return csv_files

def combine_csv_files(csv_files, output_file):
    print(f"Combining {len(csv_files)} CSV files")
    try:
        dfs = []
        for file in tqdm(csv_files, desc="Reading CSV files"):
            try:
                df = pd.read_csv(file, low_memory=False)
                if not df.empty:
                    dfs.append(df)
                else:
                    print(f"Warning: Skipping empty CSV file: {file}")
            except Exception as e:
                print(f"Error processing {file}: {e}")
        
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            combined_df.to_csv(output_file, index=False)
            print(f"Combined CSV saved to {output_file}")
            print(f"Total rows in combined CSV: {len(combined_df)}")
        else:
            print("Error: No non-empty CSV files to combine.")
    except Exception as e:
        print(f"Error combining CSV files: {e}")

if __name__ == "__main__":
    input_directory = "./inputs"
    output_directory = "./output"
    combined_output = os.path.join(output_directory, "combined_output.csv")

    os.makedirs(output_directory, exist_ok=True)
    
    # Process PCAP files
    csv_files = process_pcap_files(input_directory, output_directory)
    
    # Combine CSV files
    if csv_files:
        combine_csv_files(csv_files, combined_output)
    else:
        print("No CSV files were generated from PCAP files.")

Processing PCAP files
No CSV files were generated from PCAP files.
